In [ ]:
%load_ext autoreload
%autoreload 2
# %env CUDA_DEVICE_ORDER=PCI_BUS_ID
# %env CUDA_VISIBLE_DEVICES=1
# %env CUDA_LAUNCH_BLOCKING=1
%env MKL_NUM_THREADS=1
%env OMP_NUM_THREADS=1
%env NUMEXPR_NUM_THREADS=1
%env MKL_DEBUG_CPU_TYPE=5

# Set up the gym and wrap a monitor around it that will periodically record movies as it learns

In [ ]:
import gym
# from gym_recording.wrappers import TraceRecordingWrapper
from surface_seg.envs.mcs_env import MCSEnv
from surface_seg.utils.callback_simple import Callback
from tensorforce.execution import Runner
import gym.wrappers
import numpy as np
import tensorforce
import copy
import tensorflow as tf

In [ ]:
timesteps=500

In [ ]:
def setup_env(recording=True):
    
    # Set up gym
    MCS_gym = MCSEnv(observation_fingerprints=False, 
                     observation_forces=False,
                     permute_seed=42,
                     save_dir = './result_trpo/test/',
                     timesteps = timesteps,
                     save_every = 2,
                     plot_every = 2,
                    )
    
    if recording:
    # Wrap the gym to provide video rendering every 50 steps
        MCS_gym = gym.wrappers.Monitor(MCS_gym, 
                                         "./vid_trpo/test", 
                                         force=True,
                                        video_callable = lambda episode_id: (episode_id+1)%30==0) #every 50, starting at 51
    
    #Convert gym to tensorforce environment
    env = tensorforce.environments.OpenAIGym(MCS_gym,
                                         max_episode_timesteps=timesteps,
                                         visualize=False)
    
    return env
    
env = setup_env().environment.env
print('initial energy', env.initial_energy)
print('thermal energy', env.thermal_energy)
n = 3
print('%dKT' %n, n * env.thermal_energy)

# Set up the gym and agent in tensorforce

In [ ]:
from tensorforce.agents import Agent

agent = Agent.create(
    agent='trpo', 
    environment=setup_env(), 
    batch_size=1, 
    learning_rate=1e-3,
    memory = 40000,
    max_episode_timesteps = timesteps,
    exploration=dict(
        type='decaying', unit='timesteps', decay='exponential',
        initial_value=0.3, decay_steps=3000000, decay_rate=0.5
    ),
    recorder = dict(
        directory = './recorder/test', frequency=1), #required for recording states and actions
    summarizer = dict(
        directory = 'tb/test', labels='all', frequency=1, #Tensorboard summarizer
    )
)
    

agent_spec = agent.spec

# Run the DRL method in parallel (multiple environments)

In [ ]:
callback = Callback('./result_trpo/test').episode_finish

num_parallel = 32
runner = Runner(
    agent=agent_spec,
    environments=[setup_env(recording=True) for _ in range(num_parallel)],
    num_parallel=num_parallel,
    remote='multiprocessing',
    max_episode_timesteps=timesteps,
)

runner.run(num_episodes=num_parallel*2, callback=callback, callback_episode_frequency=1)
# runner.run(num_episodes=100, evaluation=True)
runner.close()